In [1]:
get angular rate to zero. all satelilte have imu, that gies gyroscope data (spin rate

LoadError: syntax: extra token "angular" after end of expression

In [2]:
#take those measurements as our results. reward is functino of out spin rate.can make it more
#advanced by adding a power cost to do the movement. by modifying reward function. reward(time, power cost, and spin change)
inputs: state data will be magentometer data and current spin rate. tor
#input: current change in magnetic fileld at snapshot (Bdot) and urrent spin on satellite wo
action: is moment im creating and amount of time ill create it forstart w t as a constant at .001
then use that to create the torque, which will tell me the new spin spped wf

LoadError: syntax: extra token "data" after end of expression

In [3]:
import Pkg
Pkg.add("Plotly")
using Pkg
Pkg.add("Interpolations")
using Interpolations
using Plotly
using LinearAlgebra

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


WebIO._IJuliaInit()

In [4]:
function findnearestindex(minimum,maximum,numsteps,value)
    normalized_value = max(0 , min(((value - minimum)/(maximum - minimum)), 0.999999))
    new_index = floor(Int, 1 + (normalized_value * numsteps))
    return new_index
end

findnearestindex (generic function with 1 method)

In [5]:
mutable struct Sarsa
   S
    A
    Y
    Q
    alpha
    L
end

function update!(model::Sarsa, s, a, r, sprime)
    if model.L != nothing
        Y, Q, alpha, L = model.Y, model.Q, model.alpha, model.L
#         print("L\n")
#         print(L)
#         print("s")
#         print(s)
#         print("a")
#         print(a)
#         print("Q[s,a]")
#         print(Q[s,a])
#         print("Q[L.s, L.a]")
#         print(Q[L.s, L.a])
        model.Q[L.s, L.a] += alpha * (L.r + Y * Q[s,a] - Q[L.s, L.a])
    end
    model.L = (s=s, a=a, r=r)
    return model
end



update! (generic function with 1 method)

In [6]:
data = []
#state space, min max spin rates (omegaxyz) and magentic field (bx,by,bz)
#action is the moment
#there is julia function where i can input min, max, and step sizes


num_states = 100




#new transitions are random mag fields
# -3 to 3, 100 total divisions for spin rate
# -7 t8 10^-5 to 7 *10^-5
#action is moment, between -1 and 1 for each dimension
function beginLearning(start_state, states, actions)
    #time is constant
    Q = zeros(size(states)[1], size(actions)[1])
    epsilon = .2
    first_action = rand(actions)
    first_action_index = findfirst(isequal(first_action), actions)
    first_state_index = findfirst(isequal(start_state), states)
    sarsa_struct = Sarsa(size(states)[1], size(actions)[1], .95, Q, .07, (s=first_state_index, a=first_action_index, r=0))
    #init_action = get_action(start_state)
    #newState = transition(start_state, init_action)
    w0 = start_state[2]
    curr_state = start_state
    ten_most_recent = Array{Tuple{Float64, Float64, Float64}}(undef,10)
    recent_index = 1
    range_actions = collect(range(start = (-1), length=num_divisions_for_state, stop=(1)))
    spin_range = collect(range(start = (-3), length=num_divisions_for_state, stop=(3)))
    
    i_max = 10^7
    for i=1:i_max
        ten_most_recent[recent_index] = curr_state[2]
        if i%100000 == 0
           print(i,"/",i_max,"\n") 
        end
        
        #next_state = transition(curr_state, action)
        #bdot
        if (rand() < epsilon)
            vectorized_B = [i for i in curr_state[1]] 
            vectorized_w = [i for i in curr_state[2]]
            #interp now

            exact_action = BDot_solution(vectorized_B, vectorized_w; ω_0=w0)
            #findnearestindex(min,max,numsteps,value)
            nearest_action_x_index = findnearestindex(-1,1,num_divisions_for_state,exact_action[1])
            nearest_action_y_index = findnearestindex(-1,1,num_divisions_for_state,exact_action[2])
            nearest_action_z_index = findnearestindex(-1,1,num_divisions_for_state,exact_action[3])
            #action_interper = scale(interpolate(actions, BSpline(Constant())), range(start = (-1), length=num_divisions_for_state, stop=(1)))

            actionx = range_actions[nearest_action_x_index]
            actiony = range_actions[nearest_action_y_index]
            actionz = range_actions[nearest_action_z_index]
            action = (actionx, actiony, actionz)
            action_index = findfirst(isequal(action), actions)
            vectorized_action = [i for i in action]


            new_spin = Determine_ω(vectorized_B, vectorized_w, vectorized_action; time_step=1, I=I_sapling, time=0)
            nearest_spin_x_index = findnearestindex(-3, 3, num_divisions_for_state, new_spin[1])
            nearest_spin_y_index = findnearestindex(-3, 3, num_divisions_for_state, new_spin[2])
            nearest_spin_z_index = findnearestindex(-3, 3, num_divisions_for_state, new_spin[3])

            new_spin_x = spin_range[nearest_spin_x_index]
            new_spin_y = spin_range[nearest_spin_y_index]
            new_spin_z = spin_range[nearest_spin_z_index]
            next_state = (curr_state[1], (new_spin_x, new_spin_y, new_spin_z))
            

            curr_state_index = findfirst(isequal(curr_state), states)
            next_state_index = findfirst(isequal(next_state), states)
        #random
        else
            next_state = rand(states)
            action = rand(actions)
            action_index = findfirst(isequal(action), actions)
            curr_state_index = findfirst(isequal(curr_state), states)
            next_state_index = findfirst(isequal(next_state), states)
        end

        update!(sarsa_struct, curr_state_index, action_index, Calculate_Reward(curr_state[2], next_state[2], w0), next_state_index)
        curr_state = next_state
        recent_index += 1
        if (recent_index > 10)
           recent_index = 1 
        end
    end
    return Q
end

#collect(Iterators.product(-2:2, 3:5, 1:2))

num_divisions_for_state = 10
range_of_magfields = collect(range(start = (-7*10^(-5)), length=num_divisions_for_state, stop=(7*10^(-5))))
range_of_spinrates = collect(range(start = (-3), length=num_divisions_for_state, stop=(3)))
states = []
b_norm = 7*10^-5
b_const = (1,0,0)
b = (b_const[1] * b_norm, b_const[2] * b_norm, b_const[3] * b_norm)
#print(b)
for omegax in range_of_spinrates
   for omegay in range_of_spinrates
       for omegaz in range_of_spinrates
           push!(states, (b, (omegax, omegay, omegaz))) 
        end
    end
end
actions = []
range_actions = collect(range(start = (-1), length=num_divisions_for_state, stop=(1)))
for x in range_actions
   for y in range_actions
       for z in range_actions
            push!(actions, (x,y,z))
        end
    end
end
#

#itp = interpolate(A, BSpline(Cubic(Line(OnGrid()))))


Q = beginLearning(rand(states),states,actions)

#collect(Iterators.product((range_of_magfields, range_of_magfields)))


LoadError: UndefVarError: BDot_solution not defined

In [7]:
policy = mapslices(argmax, Q, dims=2)
range_of_spinrates = collect(range(start = (-3), length=num_divisions_for_state, stop=(3)))

function simulate(policy, starting_state, states, actions; 
                    time_initial = 0, time_span = 60*2, time_step=1)
    time_hist = [time_initial]
    state_hist = [starting_state]
    time_end = time_initial + time_span
    time = time_initial
    curr_state = starting_state
    while time < time_end
        time = time_hist[end]
        curr_state = state_hist[end]
        
        # find coresponding discretized state
        w_x = findnearestindex(-3, 3, num_divisions_for_state, curr_state[2][1])
        w_y = findnearestindex(-3, 3, num_divisions_for_state, curr_state[2][2])
        w_z = findnearestindex(-3, 3, num_divisions_for_state, curr_state[2][3])
        discretized_state = (curr_state[1], (range_of_spinrates[w_x], range_of_spinrates[w_y], range_of_spinrates[w_z]))
        curr_state_index = findfirst(isequal(discretized_state), states)
        # find action         
        action_index = policy[curr_state_index]
        action = actions[action_index]
        # determine new state
        vectorized_B = [i for i in discretized_state[1]] 
        vectorized_w = [i for i in curr_state[2]]
        vectorized_action = [i for i in action]
        new_spin = Determine_ω(vectorized_B, vectorized_w, vectorized_action; time_step=time_step, I=I_sapling, time=time)
        push!(time_hist, time+time_step)
        push!(state_hist, (curr_state[1],(new_spin[1], new_spin[2], new_spin[3])))        
    end
    return(time_hist, state_hist)
end

LoadError: UndefVarError: Q not defined

In [8]:
time_hist, state_hist = simulate(policy, rand(states), states, actions; time_initial = 0, time_span = 60*2, time_step=1)

LoadError: UndefVarError: simulate not defined

In [9]:
# functions for RL Model
I_sapling = [0.00165567, 0.00167717, 0.00178716] # kg*m^2
ω_0 = [deg2rad(3.),deg2rad(3.),deg2rad(3.)] # rad/sec
b_norm = 7*10^-5 # Tesla
b_const = [1,0,0]

"""
    Euler Equations with applied torque conditions used with numerical propogator
    to propogate angular velocity
"""
function EulerEquations(ω, time, I, M)
    # calculate dω/dt
    dωdt = [(M[1]-((I[3]-I[2])*ω[2]*ω[3]))/I[1], (M[2]-((I[1]-I[3])*ω[3]*ω[1]))/I[2], (M[3]-((I[2]-I[1])*ω[1]*ω[2]))/I[3]]
end

# RKF45 constants (to be stored in seperate file)
const A2 = 2. / 9.
const A3 = 1. / 3.
const A4 = 3. / 4.
const A5 = 1.
const A6 = 5. / 6.
const B21 = 2. / 9.
const B31 = 1. / 12.
const B41 = 69. / 128.
const B51 = -17. / 12.
const B61 = 65. / 432.
const B32 = 1. / 4.
const B42 = -243. / 128.
const B52 = 27. / 4.
const B62 = -5. / 16.
const B43 = 135. / 64.
const B53 = -27. / 5.
const B63 = 13. / 16.
const B54 = 16. / 15.
const B64 = 4. / 27.
const B65  = 5. / 144.
const CH1 = 47. / 450.
const CH2 = 0.
const CH3 = 12. / 25.
const CH4 = 32. / 225.
const CH5 = 1. / 30.
const CH6 = 6. / 25.
const CT1 = -1. / 150.
const CT2 = 0.
const CT3 = 3. / 100.
const CT4 = -16. / 75.
const CT5 = -1. / 20.
const CT6 = 6. / 25.

# https://en.wikipedia.org/wiki/Runge%E2%80%93Kutta%E2%80%93Fehlberg_method
function RK45(x, time, step, dxdt)
    # compute k1 through k6
    k1 = step * dxdt(x, time)
    modifier = B21 * k1
    k2 = step * dxdt(x + modifier, time + (A2 * step))
    modifier = B31 * k1 + B32 * k2
    k3 = step * dxdt(x + modifier, time + (A3 * step))
    modifier = B41 * k1 + B42 * k2 + B43 * k3
    k4 = step * dxdt(x + modifier, time + (A4 * step))
    modifier = B51 * k1 + B52 * k2 + B53 * k3 + B54 * k4
    k5 = step * dxdt(x + modifier, time + (A5 * step))
    modifier = B61 * k1 + B62 * k2 + B63 * k3 + B64 * k4 + B65 * k5
    k6 = step * dxdt(x + modifier, time + (A6 * step))
    # compute weighted average
    x_new = x + CH1 * k1 + CH2 * k2 + CH3 * k3 + CH4 * k4 + CH5 * k5 + CH6 * k6
    # compute error
    error = abs.(CT1 * k1 + CT2 * k2 + CT3 * k3 + CT4 * k4 + CT5 * k5 + CT6 * k6)
    # NOTE: we can add in adaptive time steps here if we want...
    # return new x values and numerical integration error
    return(x_new, error)
end

function BDot_solution(B, ω; ω_0=ω_0)
    B_dot = cross(-ω,B)
    B_dot_body_norm = B_dot/norm(B_dot)
    M = -(1 - (norm(ω_0)-norm(ω))/norm(ω_0))*B_dot_body_norm
    return(M)
end

#random avtion/moment in nongreedy case. rand(of normal distribution give it )
function Determine_ω(B, ω, M; time_step=1, I=I_sapling, time=0)
    torque = cross(M,B)
    EulerEquations_M(ω, time) = EulerEquations(ω, time, I, torque)
    ω_new, error = RK45(ω, time, .1, EulerEquations_M)
    return ω_new
end

function Calculate_Reward(ω_prev, ω_new, ω_0)
   reward = (norm(ω_prev) - norm(ω_new))^3
   if norm(ω_new) > norm(ω_0)
        reward *= 100
   end
   return reward 
end

Calculate_Reward (generic function with 1 method)